In [1]:
import pandas as pd
import re

# Charger le fichier CSV
df = pd.read_csv('filtered_tweets_engie.csv', delimiter=';')

# Convertir la colonne 'created_at' en datetime
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce', utc=True)


# Supprimer la colonne 'name' (expliqué dans la consigne)
df = df.drop(columns=['name'])

# Supprimer les tweets où 'screen_name' est 'ENGIEpartFR'
df = df[df['screen_name'] != 'ENGIEpartFR']

# Supprimer les caractères spéciaux, les emojis et les sauts de ligne
def clean_text(text):
    # Enlever les sauts de ligne et les retours chariots
    text = text.replace('\n', ' ').replace('\r', ' ')
    # Enlever les caractères spéciaux
    text = re.sub(r'(https?://[^\s]+)','',text)
    text = re.sub(r'[^\w\s,!?;.\'\"àâäéèêëîïôöùûüç]', '', text)  # Garde les lettres, chiffres, accents et certains signes de ponctuation
    return text

df['full_text'] = df['full_text'].apply(clean_text)

# Supprimer les messages successifs du même 'screen_name'
df['prev_screen_name'] = df['screen_name'].shift(1)  # Créer une colonne pour le 'screen_name' précédent
df['is_duplicate'] = df['screen_name'] == df['prev_screen_name']  # Marquer les messages successifs

# Filtrer les doublons successifs
df_filtered = df[~df['is_duplicate']]  # Garder uniquement les tweets qui ne sont pas des doublons successifs

# Supprimer la colonne temporaire
df_filtered.drop(columns=['prev_screen_name', 'is_duplicate'], inplace=True)

# Renommer les 'id' pour commencer à 1 et incrémenter de 1 à chaque ligne
df_filtered['id'] = range(1, len(df_filtered) + 1)

# Sauvegarder le DataFrame nettoyé dans un nouveau fichier CSV
df_filtered.to_csv('filtered_tweets_engie_cleaned.csv', index=False, sep=';')

# Afficher les premières lignes du DataFrame nettoyé
print(df_filtered.head())

   id      screen_name                created_at  \
0   1       gptournier 2023-11-16 15:13:18+00:00   
1   2       jouanetwan 2023-11-26 07:34:34+00:00   
2   3  vince_thouvenin 2023-12-21 14:27:08+00:00   
3   4   BiduleAnatheme 2023-12-28 21:32:58+00:00   
4   5  vince_thouvenin 2023-12-29 10:08:10+00:00   

                                           full_text  
0  ENGIEpartFR n6 mois dattente et tjs aucune rép...  
1  Bonjour ENGIEpartSAV , lappli monpilotageelec ...  
2  ENGIEpartFR mon syndic de copropriété sergic e...  
3  ENGIEpartSAV vous envisagez de vous occuper de...  
4  ENGIEpartSAV retour de votre technicien "vous ...  


C:\Users\ludov\AppData\Local\Temp\ipykernel_25960\2014227523.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['prev_screen_name', 'is_duplicate'], inplace=True)
C:\Users\ludov\AppData\Local\Temp\ipykernel_25960\2014227523.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['id'] = range(1, len(df_filtered) + 1)


In [2]:
df_filtered["date"] = df_filtered["created_at"].dt.date
df_filtered["semaine"] = df_filtered["created_at"].dt.to_period("W").astype(str)
df_filtered["mois"] = df_filtered["created_at"].dt.to_period("M").astype(str)
df_filtered["annee"] = df_filtered["created_at"].dt.to_period("Y").astype(str)

tweets_par_jour = df_filtered["date"].value_counts().sort_index()
tweets_par_semaine = df_filtered["semaine"].value_counts().sort_index()
tweets_par_mois = df_filtered["mois"].value_counts().sort_index()
tweets_par_annee = df_filtered["annee"].value_counts().sort_index()

print("\nNombre de tweets par jour :\n", tweets_par_jour.to_frame("Tweets"))
print("\nNombre de tweets par semaine :\n", tweets_par_semaine.to_frame("Tweets"))
print("\nNombre de tweets par mois :\n", tweets_par_mois.to_frame("Tweets"))
print("\nNombre de tweets par année :\n", tweets_par_annee.to_frame("Tweets"))


# Comptage des mentions des comptes Engie
engie_accounts = ["ENGIEgroup", "ENGIEpartFR", "ENGIEpartSAV"]
mentions = {account: df_filtered["full_text"].str.contains(account, case=False, na=False).sum() for account in engie_accounts}

# Afficher les résultats des mentions
print("\nNombre de tweets mentionnant les comptes Engie :")
for account, count in mentions.items():
    print(f"{account}: {count}")


Nombre de tweets par jour :
             Tweets
date              
2023-11-16       1
2023-11-26       1
2023-12-21       1
2023-12-28       1
2023-12-29       3
...            ...
2025-02-24       4
2025-02-25       3
2025-02-26       3
2025-03-03       5
2025-03-04       1

[286 rows x 1 columns]

Nombre de tweets par semaine :
                        Tweets
semaine                      
2023-11-13/2023-11-19       1
2023-11-20/2023-11-26       1
2023-12-18/2023-12-24       1
2023-12-25/2023-12-31       5
2024-01-01/2024-01-07      12
...                       ...
2025-02-03/2025-02-09       3
2025-02-10/2025-02-16       8
2025-02-17/2025-02-23      10
2025-02-24/2025-03-02      10
2025-03-03/2025-03-09       6

[66 rows x 1 columns]

Nombre de tweets par mois :
          Tweets
mois           
2023-11       2
2023-12       6
2024-01      58
2024-02      54
2024-03      47
2024-04      40
2024-05      36
2024-06      20
2024-07      28
2024-08      27
2024-09      20
2024-10      36

C:\Users\ludov\AppData\Local\Temp\ipykernel_25960\459880033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["date"] = df_filtered["created_at"].dt.date
C:\Users\ludov\AppData\Local\Temp\ipykernel_25960\459880033.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_filtered["semaine"] = df_filtered["created_at"].dt.to_period("W").astype(str)
C:\Users\ludov\AppData\Local\Temp\ipykernel_25960\459880033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-